In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from linearmodels import PanelOLS
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import ResidualsPlot
from scipy import stats

In [2]:
joint = pd.read_excel('joint_snap.xlsx')

In [3]:
#convert the month column into correct form
joint['Month'] = pd.to_datetime(joint['Month'])
age = (joint['Month'] - joint['Inception Date'])/np.timedelta64(1,'M')
age = [int(i) for i in age]
joint['Age'] = age
#joint.dropna(subset=["esg_score"], inplace=True)

#convert the esb_l from string to float
esb_l = pd.DataFrame({'Equity Style Box (Long)':['Large Value', 'Large Growth', 'Large Blend', 'Mid Value', 'Mid Growth', 'Mid Blend','Small Value','Small Growth','Small Blend'], 'esb_l': [1, 2, 3, 4, 5, 6, 7, 8, 9]})
joint_new = joint.join(esb_l.set_index('Equity Style Box (Long)'), on='Equity Style Box (Long)')

joint_new = joint_new.rename(columns={"Fund Sustainability": "esg_score"})

In [12]:
size_median = joint_new['size_log'].median()
size_less = joint_new[joint_new['size_log']<size_median]
size_more = joint_new[joint_new['size_log']>size_median]

In [15]:
#catagorize the month - 2012/01 to 2019/08, totoally 92 months
month_less = pd.Categorical(size_less.Month)
month_more = pd.Categorical(size_more.Month)
#use the 'Month' to divide the dataset, the index is Fund ID
df_less = size_less.set_index(['FundId', 'Month'])
df_less['Month'] = month_less

df_more = size_more.set_index(['FundId', 'Month'])
df_more['Month'] = month_more

In [19]:
#capm Performance summary
exog_vars = ['esg_score','size_log', 'Age']

exog_less = sm.add_constant(df_less[exog_vars])
exog_more = sm.add_constant(df_more[exog_vars])

mod_less = PanelOLS(df_less.capm_Performance, exog_less, entity_effects=True, time_effects=True)
mod_more = PanelOLS(df_more.capm_Performance, exog_more, entity_effects=True, time_effects=True)

res_less = mod_less.fit()
res_more = mod_more.fit()

/home/yuxuan/anaconda3/lib/python3.7/site-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [20]:
print(res_less)
print(res_more)

                          PanelOLS Estimation Summary                           
Dep. Variable:       capm_Performance   R-squared:                        0.0002
Estimator:                   PanelOLS   R-squared (Between):             -0.2849
No. Observations:               68390   R-squared (Within):              -0.0006
Date:                Tue, Apr 07 2020   R-squared (Overall):             -0.0315
Time:                        10:47:59   Log-likelihood                 -1.58e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      4.0875
Entities:                        2347   P-value                           0.0065
Avg Obs:                       29.139   Distribution:                 F(3,65950)
Min Obs:                       1.0000                                           
Max Obs:                       91.000   F-statistic (robust):             4.0875
                            